# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments                Customer  \
0  Mar 17 2023  2:40PM  259518        19       N17997  NAPP Technologies, LLC   
1  Mar 17 2023  2:38PM  259517        10        U1244        Uniderm USA, Inc   
2  Mar 17 2023  2:30PM  259516        19  ADV80013982     AdvaGen Pharma, Ltd   
3  Mar 17 2023  2:30PM  259516        19  ADV80013983     AdvaGen Pharma, Ltd   
4  Mar 17 2023  2:30PM  259516        19  ADV80013984     AdvaGen Pharma, Ltd   
5  Mar 17 2023  2:30PM  259516        19  ADV80013985     AdvaGen Pharma, Ltd   
6  Mar 17 2023  2:30PM  259516        19  ADV80013991     AdvaGen Pharma, Ltd   
7  Mar 17 2023  2:30PM  259516        19  ADV80013992     AdvaGen Pharma, Ltd   
8  Mar 17 2023  2:30PM  259516        19  ADV80013980     AdvaGen Pharma, Ltd   
9  Mar 17 2023  2:30PM  259516        19  ADV80013981     AdvaGen Pharma, Ltd   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4      Executing  
5       Released  
6      Executing  
7       Released  
8      Executing  
9      Executing

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
16  259515       Released          1
17  259516      Executing          4
18  259516       Released          4
19  259517       Released          1
20  259518       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
259513                NaN        NaN       1.0
259515                NaN        NaN       1.0
259516                NaN        4.0       4.0
259517                NaN        NaN       1.0
259518                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259473               11.0        4.0       8.0
259478               20.0        2.0       0.0
259494                0.0        0.0       1.0
259496                0.0        0.0       6.0
259497                0.0        1.0       6.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259473                 11          4         8
259478                 20          2         0
259494                  0          0         1
259496                  0          0         6
259497                  0          1         6

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               259473         11          4         8
1               259478         20          2         0
2               259494          0          0         1
3               259496          0          0         6
4               259497          0          1         6

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               259473        11         4        8
1               259478        20         2         
2               259494                            1
3               259496                            6
4               259497                   1        6

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                Customer
0   Mar 17 2023  2:40PM  259518        19  NAPP Technologies, LLC
1   Mar 17 2023  2:38PM  259517        10        Uniderm USA, Inc
2   Mar 17 2023  2:30PM  259516        19     AdvaGen Pharma, Ltd
10  Mar 17 2023  2:24PM  259515        19    Emersa Waterbox, LLC
11  Mar 17 2023  2:10PM  259513        10               Hush Hush
12  Mar 17 2023  2:08PM  259512        19    Emersa Waterbox, LLC
13  Mar 17 2023  1:57PM  259510        19    Emersa Waterbox, LLC
14  Mar 17 2023  1:55PM  259509        22       NBTY Global, Inc.
15  Mar 17 2023  1:36PM  259506        19    Emersa Waterbox, LLC
16  Mar 17 2023  1:29PM  259503        19    Emersa Waterbox, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                Customer Completed  \
0  Mar 17 2023  2:40PM  259518        19  NAPP Technologies, LLC             
1  Mar 17 2023  2:38PM  259517        10        Uniderm USA, Inc             
2  Mar 17 2023  2:30PM  259516        19     AdvaGen Pharma, Ltd             
3  Mar 17 2023  2:24PM  259515        19    Emersa Waterbox, LLC             
4  Mar 17 2023  2:10PM  259513        10               Hush Hush             
5  Mar 17 2023  2:08PM  259512        19    Emersa Waterbox, LLC             
6  Mar 17 2023  1:57PM  259510        19    Emersa Waterbox, LLC             
7  Mar 17 2023  1:55PM  259509        22       NBTY Global, Inc.             
8  Mar 17 2023  1:36PM  259506        19    Emersa Waterbox, LLC             
9  Mar 17 2023  1:29PM  259503        19    Emersa Waterbox, LLC             

  Executing Released  
0                  1  
1                  1  
2         4        4  
3                  1  
4                  1  
5                  1  
6                  1  
7                  1  
8                  1  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                Customer Released  \
0  Mar 17 2023  2:40PM  259518        19  NAPP Technologies, LLC        1   
1  Mar 17 2023  2:38PM  259517        10        Uniderm USA, Inc        1   
2  Mar 17 2023  2:30PM  259516        19     AdvaGen Pharma, Ltd        4   
3  Mar 17 2023  2:24PM  259515        19    Emersa Waterbox, LLC        1   
4  Mar 17 2023  2:10PM  259513        10               Hush Hush        1   
5  Mar 17 2023  2:08PM  259512        19    Emersa Waterbox, LLC        1   
6  Mar 17 2023  1:57PM  259510        19    Emersa Waterbox, LLC        1   
7  Mar 17 2023  1:55PM  259509        22       NBTY Global, Inc.        1   
8  Mar 17 2023  1:36PM  259506        19    Emersa Waterbox, LLC        1   
9  Mar 17 2023  1:29PM  259503        19    Emersa Waterbox, LLC        1   

  Executing Completed  
0                      
1                      
2         4            
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                Customer Released  \
0  Mar 17 2023  2:40PM  259518        19  NAPP Technologies, LLC        1   
1  Mar 17 2023  2:38PM  259517        10        Uniderm USA, Inc        1   
2  Mar 17 2023  2:30PM  259516        19     AdvaGen Pharma, Ltd        4   
3  Mar 17 2023  2:24PM  259515        19    Emersa Waterbox, LLC        1   
4  Mar 17 2023  2:10PM  259513        10               Hush Hush        1   

  Executing Completed  
0                      
1                      
2         4            
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                Customer  Released  \
0  Mar 17 2023  2:40PM  259518        19  NAPP Technologies, LLC       1.0   
1  Mar 17 2023  2:38PM  259517        10        Uniderm USA, Inc       1.0   
2  Mar 17 2023  2:30PM  259516        19     AdvaGen Pharma, Ltd       4.0   
3  Mar 17 2023  2:24PM  259515        19    Emersa Waterbox, LLC       1.0   
4  Mar 17 2023  2:10PM  259513        10               Hush Hush       1.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        4.0        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                Customer  Released  \
0  Mar 17 2023  2:40PM  259518        19  NAPP Technologies, LLC       1.0   
1  Mar 17 2023  2:38PM  259517        10        Uniderm USA, Inc       1.0   
2  Mar 17 2023  2:30PM  259516        19     AdvaGen Pharma, Ltd       4.0   
3  Mar 17 2023  2:24PM  259515        19    Emersa Waterbox, LLC       1.0   
4  Mar 17 2023  2:10PM  259513        10               Hush Hush       1.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        4.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1038023      14.0        1.0        0.0
15          259473       8.0        4.0       11.0
18          259499       0.0        1.0        0.0
19         2076058      10.0        6.0       20.0
22          519003       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1038023      14.0        1.0        0.0
1        15   259473       8.0        4.0       11.0
2        18   259499       0.0        1.0        0.0
3        19  2076058      10.0        6.0       20.0
4        22   519003       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      14.0        1.0        0.0
1        15       8.0        4.0       11.0
2        18       0.0        1.0        0.0
3        19      10.0        6.0       20.0
4        22       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   14.0
1        15  Released    8.0
2        18  Released    0.0
3        19  Released   10.0
4        22  Released    2.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   18    19   22
Status                               
Completed   0.0  11.0  0.0  20.0  0.0
Executing   1.0   4.0  1.0   6.0  0.0
Released   14.0   8.0  0.0  10.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   18    19   22
0          Completed   0.0  11.0  0.0  20.0  0.0
1          Executing   1.0   4.0  1.0   6.0  0.0
2           Released  14.0   8.0  0.0  10.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   18    19   22
0  Completed   0.0  11.0  0.0  20.0  0.0
1  Executing   1.0   4.0  1.0   6.0  0.0
2   Released  14.0   8.0  0.0  10.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()